In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input
from IPython.display import display, Image
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator


2023-11-24 02:34:30.282875: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Define the path to the dataset folders
happy_folder = "pets_facial_expression_dataset/happy"
sad_folder = "pets_facial_expression_dataset/Sad"
angry_folder = "pets_facial_expression_dataset/Angry"

# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (128, 128))  # Resize to a fixed size for the model
            images.append(img)
    return images

# Load images and labels for each emotion
happy_images = load_images_from_folder(happy_folder)
sad_images = load_images_from_folder(sad_folder)
angry_images = load_images_from_folder(angry_folder)

# Create labels for each emotion category
happy_labels = [0] * len(happy_images)
sad_labels = [1] * len(sad_images)
angry_labels = [2] * len(angry_images)

# Concatenate images and labels
X = np.array(happy_images + sad_images + angry_images )
y = np.array(happy_labels + sad_labels + angry_labels )

# Normalize pixel values to range [0, 1]
X = X.astype('float32') / 255.0

# One-hot encode the labels
y = to_categorical(y, 3)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#### Load the dataset  (Optional if integrate another model)

In [3]:
# train_gen=ImageDataGenerator(rescale=1./255)
# train_data=train_gen.flow_from_directory('pets_facial_expression_dataset/master/train',target_size=(128,128),batch_size=32,class_mode='categorical',shuffle=True) 
# val_gen=ImageDataGenerator(rescale=1./255)
# val_data=val_gen.flow_from_directory('pets_facial_expression_dataset/master/valid',target_size=(128,128),batch_size=1,class_mode='categorical',shuffle=False)
# test_gen=ImageDataGenerator(rescale=1./255)
# test_data=test_gen.flow_from_directory('pets_facial_expression_dataset/master/test',target_size=(128,128),batch_size=1,class_mode='categorical',shuffle=False)

#### Load the pre-trained model (Optional if integrate another model)

In [4]:
# Load the pre-trained model
pretrained_animal_classifier = load_model("animal_classifier_model.h5")

# Freeze the layers of the pre-trained model
for layer in pretrained_animal_classifier.layers:
    layer.trainable = False

# Take the output from the second-to-last layer of the classifier_model
# This assumes that your classifier_model's last four layers are demse, dropout, dense, flatten
x = pretrained_animal_classifier.layers[-4].output  

# Flatten and add dense layers
x = Dense(128, activation='relu', name="face")(x)
x = Dropout(0.5)(x) # Add dropout for regularization -> 62.5% accuracy
x = Dense(8, activation='relu', name="species")(x) # Softmax for probability distribution
x = Dense(3, activation='softmax', name="emotion")(x)  # Assuming 4 classes for emotion recognition

# Create the model
emotion_model = Model(inputs=pretrained_animal_classifier.input, outputs=x)

# Print the model summary
print(emotion_model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856 

In [5]:
# compile the model
emotion_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
# Calculate class weights
total_samples = len(y_train)
class_weights = {0: total_samples / np.sum(y_train[:, 0]), 
                 1: total_samples / np.sum(y_train[:, 1]), 
                 2: total_samples / np.sum(y_train[:, 2])}

# fit the model while ensuring avoiid overfitting by using early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='auto', restore_best_weights=True)
history = emotion_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, class_weight=class_weights, callbacks=[early_stopping])

# Evaluate the model on the training set
losstr, accuracytr = emotion_model.evaluate(X_train, y_train, verbose=0)
print("Train loss:", losstr)
print("Train accuracy:", accuracytr)


# Evaluate the model on the test set
loss, accuracy = emotion_model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

# early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto', restore_best_weights=True)
# # fit the model
# history = emotion_model.fit(train_data,epochs=35, batch_size=32, validation_data=val_data, callbacks=[early_stopping])

# pred=emotion_model.predict(test_data).argmax(axis=1)

# from sklearn.metrics import classification_report

# print(classification_report(pred ,test_data.classes ))




Epoch 1/50
17/17 [==============================] - 3s 142ms/step - loss: 3.2245 - accuracy: 0.4500 - val_loss: 1.0708 - val_accuracy: 0.4333
Epoch 2/50
17/17 [==============================] - 2s 113ms/step - loss: 2.9469 - accuracy: 0.4926 - val_loss: 1.0549 - val_accuracy: 0.4167
Epoch 3/50
17/17 [==============================] - 2s 115ms/step - loss: 2.7745 - accuracy: 0.5222 - val_loss: 0.9848 - val_accuracy: 0.4333
Epoch 4/50
17/17 [==============================] - 2s 113ms/step - loss: 2.7970 - accuracy: 0.5333 - val_loss: 1.0106 - val_accuracy: 0.4167
Epoch 5/50
17/17 [==============================] - 2s 113ms/step - loss: 2.5771 - accuracy: 0.5630 - val_loss: 0.9722 - val_accuracy: 0.5000
Epoch 6/50
17/17 [==============================] - 2s 118ms/step - loss: 2.4843 - accuracy: 0.6278 - val_loss: 0.9424 - val_accuracy: 0.4833
Epoch 7/50
17/17 [==============================] - 2s 112ms/step - loss: 2.3783 - accuracy: 0.6352 - val_loss: 0.9510 - val_accuracy: 0.5000
Epoch 

In [ ]:
# Save the model
emotion_model.save("pet_facial_emotion_model.h5")

#### No Loading the other model

In [ ]:
# Build the CNN emotion_model
emotion_model = Sequential()

# Add the rest of the layers
emotion_model.add(Conv2D(64, (3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(64, (3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, (3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(128, (3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(512, (3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(512, (3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(512, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(3, activation='softmax'))

print(emotion_model.summary())
# Compile the model with class weights
emotion_model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

In [ ]:
# Calculate class weights
total_samples = len(y_train)
class_weights = {0: total_samples / np.sum(y_train[:, 0]), 
                 1: total_samples / np.sum(y_train[:, 1]), 
                 2: total_samples / np.sum(y_train[:, 2])}

# Train the model with class weights
history = emotion_model.fit(X_train.reshape(-1, 128, 128, 1), y_train, batch_size=32, epochs=100, validation_split=0.1, class_weight=class_weights,verbose=0)

# Evaluate the model on test data
loss, accuracy = emotion_model.evaluate(X_test.reshape(-1, 128, 128, 1), y_test)
losstr, accuracytr = emotion_model.evaluate(X_train.reshape(-1, 128, 128, 1), y_train)

print(f"Test accuracy: {accuracy*100:.2f}%")
print(f"Train accuracy: {accuracytr*100:.2f}%")

# Save the trained model
emotion_model.save("pet_facial_emotion_model.h5")

### Test on Unseen Dataset

In [ ]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model("pet_facial_emotion_model.h5")

# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (128, 128))  # Resize to a fixed size for the model
            images.append(img)
    return images


# Load a custom test image
custom_test_image_path = "pets_facial_expression_dataset/Angry/35.jpg"

custom_test_image = cv2.imread(custom_test_image_path)
# custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
custom_test_image = cv2.resize(custom_test_image, (128, 128))
custom_test_image = custom_test_image.astype('float32') / 255.0

# Reshape the image to match the model input shape
custom_test_image = np.expand_dims(custom_test_image, axis=0)
custom_test_image = np.expand_dims(custom_test_image, axis=-1)

# Make predictions on the custom test image
prediction = loaded_model.predict(custom_test_image)
prediction_prob = prediction[0]

emotion_label = np.argmax(prediction[0])

# Map the predicted label to emotion class
emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry'}
predicted_emotion = emotion_classes[emotion_label]

# Print the custom test image and its predicted label
print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence [happy, sad, angry]: {prediction_prob}")

import matplotlib.pyplot as plt

#Display the custom test image using matplotlib
plt.imshow(custom_test_image[0, :, :, 0])
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

from PIL import Image
# Display the original custom test image using PIL
img_pil = Image.open(custom_test_image_path)
plt.imshow(np.array(img_pil))
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model("pet_facial_emotion_model.h5")



# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (128, 128))  # Resize to a fixed size for the model
            images.append(img)
    return images


# Load a custom test image
custom_test_image_path = "pets_facial_expression_dataset/happy/003.jpg"

custom_test_image = cv2.imread(custom_test_image_path)
# custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
custom_test_image = cv2.resize(custom_test_image, (128, 128))
custom_test_image = custom_test_image.astype('float32') / 255.0

# Reshape the image to match the model input shape
custom_test_image = np.expand_dims(custom_test_image, axis=0)
custom_test_image = np.expand_dims(custom_test_image, axis=-1)

# Make predictions on the custom test image
prediction = loaded_model.predict(custom_test_image)
prediction_prob = prediction[0]

emotion_label = np.argmax(prediction[0])

# Map the predicted label to emotion class
emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry'}
predicted_emotion = emotion_classes[emotion_label]

# Print the custom test image and its predicted label
print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence [happy, sad, angry]: {prediction_prob}")

import matplotlib.pyplot as plt

# Display the custom test image using matplotlib
plt.imshow(custom_test_image[0, :, :, 0])
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

from PIL import Image
# Display the original custom test image using PIL
img_pil = Image.open(custom_test_image_path)
plt.imshow(np.array(img_pil))
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
from keras.models import load_model
# Load the saved model
loaded_model = load_model("pet_facial_emotion_model.h5")



# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (128, 128))  # Resize to a fixed size for the model
            images.append(img)
    return images


# Load a custom test image
custom_test_image_path = "pets_facial_expression_dataset/Sad/001.jpg"

custom_test_image = cv2.imread(custom_test_image_path)
# custom_test_image = cv2.cvtColor(custom_test_image, cv2.COLOR_BGR2GRAY)
custom_test_image = cv2.resize(custom_test_image, (128, 128))
custom_test_image = custom_test_image.astype('float32') / 255.0

# Reshape the image to match the model input shape
custom_test_image = np.expand_dims(custom_test_image, axis=0)
custom_test_image = np.expand_dims(custom_test_image, axis=-1)

# Make predictions on the custom test image
prediction = loaded_model.predict(custom_test_image)
prediction_prob = prediction[0]

emotion_label = np.argmax(prediction[0])

# Map the predicted label to emotion class
emotion_classes = {0: 'happy', 1: 'sad', 2: 'angry'}
predicted_emotion = emotion_classes[emotion_label]

# Print the custom test image and its predicted label
print(f"Predicted Emotion: {predicted_emotion}")
print(f"Confidence [happy, sad, angry]: {prediction_prob}")

import matplotlib.pyplot as plt

#Display the custom test image using matplotlib
plt.imshow(custom_test_image[0, :, :, 0])
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

from PIL import Image
# Display the original custom test image using PIL
img_pil = Image.open(custom_test_image_path)
plt.imshow(np.array(img_pil))
plt.title(f"Predicted Emotion: {predicted_emotion}")
plt.axis('off')  # Hide axes
plt.show()

